In [1]:
import pandas as pd
from os import path
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.utils.fixes import loguniform    
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import LeaveOneGroupOut
import pickle
import numpy as np
import os
import json
import csv

In [2]:
DATA_DIR = "../../data"

In [3]:
EVAL_DIR = "2021-12-03"

In [4]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
chen_train.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [5]:
chen_valid = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
chen_test = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"))
chen_valid = pd.concat([chen_valid, chen_test])
chen_valid.head()

,Antibody_ID,heavy,light,Y,Unnamed: 0
2169,6ct7,EVQLVESGGGLVEPGGSLRLSCAVSGFDFEKAWMSWVRQAPGQGLQ...,SYELTQPPSVSVSPGQTARITCSGEALPMQFAHWYQQRPGKAPVIV...,0,NaN
1342,4nzu,AVSLVESGGGTVEPGSTLRLSCAASGFTFGSYAFHWVRQAPGDGLE...,DIEMTQSPSSLSASTGDKVTITCQASQDIAKFLDWYQQRPGKTPKL...,0,NaN
1728,5i8c,QEVLVQSGAEVKKPGASVKVSCRAFGYTFTGNALHWVRQAPGQGLE...,DIQLTQSPSFLSASVGDKVTITCRASQGVRNELAWYQQKPGKAPNL...,1,NaN
1729,5i8e,QEVLVQSGAEVKKPGASVKVSCRAFGYTFTGNALHWVRQAPGQGLE...,IQLTQSPSFLSASVGDKVTITCRASQGVRNELAWYQQKPGKAPNLL...,0,NaN
2114,6bb4,QVQLQQSDAELVKPGASVKISCKASGYTFTDRTIHWVKQRPEQGLE...,DVQMIQSPSSLSASLGDIVTMTCQASQDTSINLNWFQQKPGKAPKL...,0,NaN


In [6]:
print(len(chen_train), len(chen_valid))

1338 239


In [7]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [8]:
chen_similarity = pd.read_csv(path.join(DATA_DIR, "chen/distances/deduplicated_anarci_similarity.csv"), index_col=0)
chen_similarity.head()

,0,1,2,3,4,5,7,8,9,10,...,2398,2400,2401,2402,2403,2404,2405,2406,2407,2408
0,1.00000,0.77700,0.70350,0.69995,0.65085,0.67350,0.66915,0.66915,0.76100,0.71800,...,0.59250,0.63185,0.65150,0.61850,0.70070,0.60550,0.54015,0.70460,0.61020,0.62840
1,0.77700,1.00000,0.75000,0.73500,0.69315,0.70240,0.65570,0.65570,0.74785,0.79775,...,0.59600,0.65265,0.67200,0.63085,0.69350,0.60750,0.55310,0.71665,0.63920,0.63185
2,0.70350,0.75000,1.00000,0.76100,0.69315,0.68970,0.64550,0.64550,0.70850,0.72070,...,0.55975,0.60500,0.65050,0.62120,0.69350,0.58815,0.55555,0.69965,0.57580,0.60875
3,0.69995,0.73500,0.76100,1.00000,0.66160,0.68700,0.65500,0.65500,0.68315,0.71335,...,0.57800,0.63665,0.65900,0.61900,0.70700,0.59300,0.54950,0.67185,0.58200,0.61370
4,0.65085,0.69315,0.69315,0.66160,1.00000,0.63025,0.64000,0.64000,0.64965,0.68375,...,0.62485,0.64180,0.60255,0.60010,0.67985,0.61990,0.54340,0.76250,0.62135,0.63570


## Data clustering

In [9]:
clusters = pd.read_csv(path.join(DATA_DIR, "chen/clustering.csv"), index_col=0)
clusters.head()

,0
0,670
1,695
2,80
3,447
4,30


In [10]:
# unique clusters
clusters.nunique()

0    939
dtype: int64

In [11]:
chen_train = chen_train.merge(clusters, left_index=True, right_index=True).rename({"0": "cluster"}, axis=1)

In [12]:
# merge some cluters to obtain a 10-fold cross-validation
chen_train["cluster_merged"] = chen_train["cluster"]
chen_train["cluster_merged"][chen_train["cluster"] < 300] = chen_train["cluster"][chen_train["cluster"] < 300] // 30
chen_train["cluster_merged"][chen_train["cluster"] >= 300] = chen_train["cluster"][chen_train["cluster"] >= 300] // 100
chen_train["cluster_merged"].nunique()

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


10

In [13]:
chen_train["cluster_merged"].value_counts()

0    276
3    189
7    147
5    137
1    132
4    129
6    120
8    106
9     59
2     43
Name: cluster_merged, dtype: int64

In [14]:
chen_train.to_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data_w_clusters.csv"))

# Models

In [15]:
# different usage from the other models
def knn(n):
    model = NearestNeighbors(metric="precomputed")
    parameters = {'n_neighbors': [1,3,5]}
    return model, parameters, "kNN"

In [16]:
def logistic_regression(n):
    lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    parameters = {'C':loguniform(0.001, 1000), 'penalty': ["l2"], "solver": ["lbfgs", "sag"]}
    return lr, parameters, "logistic_regression"

In [17]:
def random_forest(n):
    rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')
    parameters = {'n_estimators': np.arange(1, 200, 10), 'max_depth': np.arange(1, min(50,n), 2), 
                  'max_features': np.arange(0.1, 0.75, 0.05)}
    return rf, parameters, "random_forest"

In [18]:
def multilayer_perceptron(n):
    mlp = MLPClassifier(random_state=42, max_iter=int(1000))
    parameters = {'hidden_layer_sizes': [(100,), (50,), (100, 100)], "activation": ["relu", "logistic"]}
    return mlp, parameters, "multilayer_perceptron"

In [19]:
def svm(n):
    svc = SVC(max_iter=8000, probability=True, class_weight='balanced')
    parameters = {'C': loguniform(0.001, 100), 'kernel':["linear", "rbf"], 'gamma': loguniform(1e-3, 1e0)}
    return svc, parameters, "SVM"

In [20]:
def gradient_boosting(n):
    gb = GradientBoostingClassifier(random_state=42, n_iter_no_change=70)
    parameters = {'learning_rate': loguniform(0.01, 0.5), 
                  'n_estimators': np.arange(1, 200, 10), 
                  'max_depth': np.arange(1, min(20,n), 2), 
                  'max_features': np.arange(0.1, 0.6, 0.1)}
    return gb, parameters, "gradient_boosting"

In [21]:
def output_evaluation(model_type, params, best_params, metrics, data, outpath, preprocessing=None):
    prepro = "_"+preprocessing if preprocessing is not None else ""
    filename = os.path.join(DATA_DIR, "evaluations", outpath, f"{model_type}_{data}{prepro}.json")
    out_dict = {
        "model_type": model_type,
        "data": data
    }
    out_dict["params"] = {}
    out_dict["best_params"] = {}
    for key, value in params.items():
        out_dict["params"][key] = str(value)
    for key, value in best_params.items():
        out_dict["best_params"][key] = str(value)
    out_dict["metrics"] = metrics
    out_dict["preprocessing"] = "none" if preprocessing is None else preprocessing
    
    json.dump(out_dict, open(filename, "w"))
    
    filename_sum = os.path.join(DATA_DIR, f"evaluations/{outpath}/all.csv")
    #"../evaluations/all.csv"
    line = [model_type, data, out_dict["preprocessing"], metrics["f1"], metrics["mcc"], metrics["acc"],metrics["precision"],metrics["recall"],metrics["auc"], filename]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [22]:
def train_and_eval(model_name, classifier, parameters, X_train, y_train, X_valid, y_valid, groups,
                   data_name, outpath, preprocessing=None):
    splitter = LeaveOneGroupOut()
    split = splitter.split(X_train, y_train, groups=groups)
    grid = RandomizedSearchCV(classifier, parameters, verbose=1, scoring="f1", cv=split)
    grid.fit(X_train, y_train)
    estimator = grid.best_estimator_
    best_params = grid.best_params_
    y_pred = estimator.predict(X_valid)
    prepro = "_"+preprocessing if preprocessing is not None else ""
    filename = path.join(DATA_DIR, "evaluations", outpath, "models", f"{model_name}_{data_name}{prepro}.pkl")
    with open(filename, 'wb') as f:
        pickle.dump(estimator, f)
    #pickle.dump(estimator, open(filename, "w"))
    metric_dict = {
        "f1": float(metrics.f1_score(y_valid, y_pred)),
        "acc": float(metrics.accuracy_score(y_valid, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_valid, y_pred)),
        "auc": float(metrics.roc_auc_score(y_valid, y_pred)),
        "precision": float(metrics.precision_score(y_valid, y_pred)),
        "recall": float(metrics.recall_score(y_valid, y_pred))
    }
    
    print(f"{model_name}, {data_name}")
    print(f"F1: {metric_dict['f1']}")
    print(f"MCC: {metric_dict['mcc']}")
    print(f"Accuracy: {metric_dict['acc']}")
    print(f"Precision: {metric_dict['precision']}")
    print(f"Recall: {metric_dict['recall']}")
    print(f"AUC: {metric_dict['auc']}")
    print(f"-----")
    
    output_evaluation(model_name, parameters, best_params, metric_dict, data_name, outpath, preprocessing=None)

In [23]:
def try_all(X_train, y_train, X_valid, y_valid, groups, data_name, outpath, preprocessing=None):
    n = len(y_train)
    for model_creator in [logistic_regression, random_forest, gradient_boosting, svm, multilayer_perceptron]:
    #for model_creator in [logistic_regression, random_forest]:

        classifier, params, model_label = model_creator(n)
        print("\n")
        print(f'Training model {model_label} on data {data_name} \n')
        train_and_eval(model_label, classifier, params, X_train, y_train, X_valid, y_valid, groups,
                   data_name, outpath, preprocessing)

In [24]:
def test_on_tap(model_name, x_test, y_test,
                   data_name, outpath, preprocessing=None):
    prepro = "_"+preprocessing if preprocessing is not None else ""
    filename = path.join(DATA_DIR, "evaluations", outpath, "models", f"{model_name}_{data_name}{prepro}.pkl")
    with open(filename, 'rb') as f:
        estimator = pickle.load(f)
    y_pred = estimator.predict(x_test)
    metric_dict = {
        "f1": float(metrics.f1_score(y_test, y_pred)),
        "acc": float(metrics.accuracy_score(y_test, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_test, y_pred)),
        "auc": float(metrics.roc_auc_score(y_test, y_pred)),
        "precision": float(metrics.precision_score(y_test, y_pred)),
        "recall": float(metrics.recall_score(y_test, y_pred))
    }
    filename_sum = os.path.join(DATA_DIR, f"evaluations/{outpath}/tap.csv")
    line = [model_name, data_name, prepro, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"], filename]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [25]:
def test_all(x_test, y_test, data_name, outpath, preprocessing=None):
    for model in ["logistic_regression", "random_forest", "gradient_boosting", "SVM", "multilayer_perceptron"]:
        print(f"Testing model {model} on {data_name}...")
        test_on_tap(model, x_test, y_test, data_name, outpath, preprocessing=preprocessing)

# Numbering

In [26]:
x_chen = pd.read_csv(path.join(DATA_DIR, "chen/integer_encoding/chen_integer_encoded.csv"), index_col=0)
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_test.index]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,271,272,273,274,275,276,277,278,279,280
2073,6aod,4,18,14,10,18,14,16,6,1,...,0,0,0,0,0,0,0,0,0,0
1517,4yny,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2025,5xcv,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2070,6and,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
666,2xqy,14,18,14,10,14,14,13,6,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
len(x_chen_train)

1338

In [28]:
x_tap = pd.read_csv(path.join(DATA_DIR, "tap/integer_encoding/tap_integer_encoded.csv"))
x_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,271,272,273,274,275,276,277,278,279,280
0,Abagovomab,14,18,9,10,14,4,16,6,1,...,0,0,0,0,0,0,0,0,0,0
1,Abituzumab,14,18,14,10,14,14,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2,Abrilumab,14,18,14,10,18,14,16,6,1,...,0,0,0,0,0,0,0,0,0,0
3,Actoxumab,14,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
4,Adalimumab,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0


In [29]:
try_all(x_chen_train.drop("Ab_ID", axis=1), chen_train["Y"], x_chen_valid.drop("Ab_ID", axis=1), chen_valid["Y"], chen_train["cluster_merged"], "integer_encoded", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data integer_encoded 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

logistic_regression, integer_encoded
F1: 0.40310077519379844
MCC: 0.21080214098455255
Accuracy: 0.6778242677824268
Precision: 0.325
Recall: 0.5306122448979592
AUC: 0.6232008592910849
-----


Training model random_forest on data integer_encoded 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
random_forest, integer_encoded
F1: 0.3815789473684211
MCC: 0.1649749446889334
Accuracy: 0.606694560669456
Precision: 0.2815533980582524
Recall: 0.5918367346938775
AUC: 0.6011815252416757
-----


Training model gradient_boosting on data integer_encoded 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, integer_encoded
F1: 0.14285714285714285
MCC: 0.02542701348578424
Accuracy: 0.7489539748953975
Precision: 0.23809523809523808
Recall: 0.10204081632653061
AUC: 0.5089151450053705
-----


Training model SVM on data integer_encoded 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/

SVM, integer_encoded
F1: 0.366412213740458
MCC: 0.15692479378199656
Accuracy: 0.6527196652719666
Precision: 0.2926829268292683
Recall: 0.4897959183673469
AUC: 0.592266380236305
-----


Training model multilayer_perceptron on data integer_encoded 

Fitting 10 folds for each of 6 candidates, totalling 60 fits
multilayer_perceptron, integer_encoded
F1: 0.3146067415730337
MCC: 0.16100221685529514
Accuracy: 0.7447698744769874
Precision: 0.35
Recall: 0.2857142857142857
AUC: 0.5744360902255639
-----


In [30]:
test_all(x_tap.drop("Ab_ID", axis=1), tap_data["Y"], "integer_encoded", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on integer_encoded...
Testing model random_forest on integer_encoded...
Testing model gradient_boosting on integer_encoded...
Testing model SVM on integer_encoded...
Testing model multilayer_perceptron on integer_encoded...


# PyBioMed

In [31]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/pybiomed/X_data.ftr"))
x_chen_train = x_chen.merge(chen_train[["Antibody_ID", "Y", "cluster_merged"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_valid = x_chen.merge(chen_valid[["Antibody_ID", "Y"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,19752,19753,19754,19755,19756,19757,19758,19759,Y,cluster_merged
0,1a0q,5.882,1.681,3.361,5.042,1.681,5.882,5.042,9.244,1.681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
1,1a14,6.667,2.500,4.167,5.000,1.667,2.500,7.500,11.667,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4
2,1a2y,3.448,5.172,4.310,6.034,1.724,2.586,6.034,10.345,0.862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,1a3l,5.882,3.361,2.521,3.361,1.681,5.882,3.361,13.445,0.840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4
4,1a4j,7.563,3.361,5.042,3.361,1.681,5.882,4.202,9.244,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7


In [32]:
x_tap = pd.read_feather(path.join(DATA_DIR, "tap/pybiomed/X_TAP_data.ftr"))
x_tap.head()

,0,1,2,3,4,5,6,7,8,9,...,19750,19751,19752,19753,19754,19755,19756,19757,19758,19759
0,10.084,3.361,2.521,3.361,1.681,3.361,5.882,11.765,0.840,1.681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.627,4.237,1.695,3.390,1.695,5.085,5.932,10.169,0.847,2.542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.932,1.695,0.000,5.932,1.695,5.085,6.780,10.169,0.847,1.695,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.738,6.557,4.098,5.738,1.639,3.279,4.918,11.475,0.820,3.279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.091,4.959,3.306,5.785,1.653,4.132,4.132,9.091,1.653,2.479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
try_all(x_chen_train.drop(["Ab_ID", "Y", "cluster_merged"], axis=1), x_chen_train["Y"], 
        x_chen_valid.drop(["Ab_ID", "Y"], axis=1),  x_chen_valid["Y"], 
        x_chen_train["cluster_merged"], "pybiomed", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data pybiomed 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

logistic_regression, pybiomed
F1: 0.631578947368421
MCC: 0.5280961581629033
Accuracy: 0.8242677824267782
Precision: 0.5538461538461539
Recall: 0.7346938775510204
AUC: 0.7910311493018258
-----


Training model random_forest on data pybiomed 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
random_forest, pybiomed
F1: 0.4761904761904762
MCC: 0.33078252586330364
Accuracy: 0.7698744769874477
Precision: 0.44642857142857145
Recall: 0.5102040816326531
AUC: 0.6735230934479054
-----


Training model gradient_boosting on data pybiomed 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, pybiomed
F1: 0.21428571428571427
MCC: 0.2805801206573421
Accuracy: 0.8158995815899581
Precision: 0.8571428571428571
Recall: 0.12244897959183673
AUC: 0.5585929108485499
-----


Training model SVM on data pybiomed 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/

SVM, pybiomed
F1: 0.5688073394495414
MCC: 0.44692368758317946
Accuracy: 0.803347280334728
Precision: 0.5166666666666667
Recall: 0.6326530612244898
AUC: 0.7400107411385607
-----


Training model multilayer_perceptron on data pybiomed 

Fitting 10 folds for each of 6 candidates, totalling 60 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


multilayer_perceptron, pybiomed
F1: 0.0
MCC: 0.0
Accuracy: 0.7949790794979079
Precision: 0.0
Recall: 0.0
AUC: 0.5
-----


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
test_all(x_tap, tap_data["Y"], "pybiomed", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on pybiomed...
Testing model random_forest on pybiomed...
Testing model gradient_boosting on pybiomed...
Testing model SVM on pybiomed...
Testing model multilayer_perceptron on pybiomed...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Protparam

In [37]:
x_chen = pd.read_csv(path.join(DATA_DIR, "chen/protparam/protparam_features.csv"))
x_chen.rename({"Unnamed: 0": "Ab_ID"}, axis=1, inplace=True)
x_chen_train = x_chen.merge(chen_train[["Antibody_ID", "Y", "cluster_merged"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_valid = x_chen.merge(chen_valid[["Antibody_ID", "Y"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_train.head()

,Ab_ID,aa_percent0_x,aa_percent1_x,aa_percent2_x,aa_percent3_x,aa_percent4_x,aa_percent5_x,aa_percent6_x,aa_percent7_x,aa_percent8_x,...,mol_extinct1_y,mol_extinct2_y,mw_y,gravy_y,ss_faction1_y,ss_faction2_y,ss_faction3_y,name,Y,cluster_merged
0,3okd,0.081967,0.016393,0.040984,0.049180,0.040984,0.106557,0.008197,0.016393,0.040984,...,19940,20065,12297.7329,-0.427679,0.276786,0.294643,0.205357,3okd,1,1
1,5fb8,0.049587,0.016529,0.049587,0.041322,0.024793,0.074380,0.016529,0.033058,0.074380,...,15930,16055,12046.3419,-0.344144,0.279279,0.306306,0.225225,5fb8,1,0
2,4olz,0.056000,0.032000,0.048000,0.040000,0.032000,0.088000,0.008000,0.032000,0.040000,...,21430,21555,11093.1436,-0.370297,0.306931,0.316832,0.158416,4olz,0,1
3,5i19,0.075630,0.016807,0.042017,0.042017,0.033613,0.134454,0.000000,0.025210,0.033613,...,14440,14565,11531.7166,-0.316822,0.271028,0.327103,0.158879,5i19,1,0
4,2xqb,0.096000,0.016000,0.048000,0.024000,0.040000,0.080000,0.000000,0.016000,0.048000,...,26930,27055,11672.8413,-0.599057,0.273585,0.330189,0.179245,2xqb,0,7


In [38]:
try_all(x_chen_train.drop(["Ab_ID", "name", "Y", "cluster_merged"], axis=1), x_chen_train["Y"], 
        x_chen_valid.drop(["Ab_ID", "name", "Y"], axis=1), x_chen_valid["Y"], 
        x_chen_train["cluster_merged"], "protparam", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data protparam 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/braz

logistic_regression, protparam
F1: 0.35220125786163514
MCC: 0.11327783655307368
Accuracy: 0.5690376569037657
Precision: 0.2545454545454545
Recall: 0.5714285714285714
AUC: 0.5699248120300752
-----


Training model random_forest on data protparam 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


random_forest, protparam
F1: 0.5161290322580645
MCC: 0.3712711205915234
Accuracy: 0.7489539748953975
Precision: 0.4266666666666667
Recall: 0.6530612244897959
AUC: 0.7133727175080558
-----


Training model gradient_boosting on data protparam 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, protparam
F1: 0.4
MCC: 0.3218463804170062
Accuracy: 0.8117154811715481
Precision: 0.5769230769230769
Recall: 0.30612244897959184
AUC: 0.6241138560687434
-----


Training model SVM on data protparam 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/

SVM, protparam
F1: 0.2692307692307692
MCC: -0.10099021488318363
Accuracy: 0.36401673640167365
Precision: 0.1761006289308176
Recall: 0.5714285714285714
AUC: 0.4409774436090225
-----


Training model multilayer_perceptron on data protparam 

Fitting 10 folds for each of 6 candidates, totalling 60 fits
multilayer_perceptron, protparam
F1: 0.20253164556962022
MCC: 0.057851371377758226
Accuracy: 0.7364016736401674
Precision: 0.26666666666666666
Recall: 0.16326530612244897
AUC: 0.5237379162191191
-----


In [39]:
x_tap = pd.read_csv(path.join(DATA_DIR, "tap/protparam/protparam_features_tap.csv"))
x_tap.rename({"Unnamed: 0": "Ab_ID"}, axis=1, inplace=True)
x_tap.head()

,Ab_ID,aa_percent0_x,aa_percent1_x,aa_percent2_x,aa_percent3_x,aa_percent4_x,aa_percent5_x,aa_percent6_x,aa_percent7_x,aa_percent8_x,...,instability_y,flexibility_y,isoelectric_y,mol_extinct1_y,mol_extinct2_y,mw_y,gravy_y,ss_faction1_y,ss_faction2_y,ss_faction3_y
0,Abagovomab,0.100840,0.016807,0.033613,0.033613,0.025210,0.117647,0.008403,0.016807,0.067227,...,53.693458,0.999564,7.973724,14440,14565,11556.8384,-0.257009,0.299065,0.299065,0.196262
1,Abituzumab,0.076271,0.016949,0.033898,0.050847,0.033898,0.101695,0.008475,0.025424,0.033898,...,42.514019,1.001379,8.586625,17420,17545,11762.9686,-0.452336,0.280374,0.299065,0.121495
2,Abrilumab,0.059322,0.016949,0.059322,0.050847,0.025424,0.101695,0.008475,0.016949,0.059322,...,40.151402,1.002818,7.970307,22460,22585,11548.7104,-0.335514,0.271028,0.336449,0.158879
3,Actoxumab,0.057377,0.016393,0.057377,0.032787,0.032787,0.114754,0.008197,0.032787,0.024590,...,51.517757,1.000328,8.682102,22460,22585,11530.7383,-0.260748,0.271028,0.317757,0.158879
4,Adalimumab,0.090909,0.016529,0.057851,0.041322,0.024793,0.090909,0.016529,0.024793,0.024793,...,46.585047,1.000522,9.428646,15930,16055,11664.9632,-0.402804,0.271028,0.289720,0.168224


In [40]:
test_all(x_tap.drop("Ab_ID", axis=1), tap_data["Y"], "protparam", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on protparam...
Testing model random_forest on protparam...
Testing model gradient_boosting on protparam...
Testing model SVM on protparam...
Testing model multilayer_perceptron on protparam...


# BERT Embeddings

In [41]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/bert/bert_chen_embeddings.ftr"))
x_chen_train = x_chen.merge(chen_train[["Antibody_ID", "Y", "cluster_merged"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_valid = x_chen.merge(chen_valid[["Antibody_ID", "Y"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,2040,2041,2042,2043,2044,2045,2046,2047,Y,cluster_merged
0,1a0q,-0.017142,-0.008645,0.008613,-0.046730,0.061336,-0.017951,0.020446,-0.108641,0.002343,...,-0.037837,-0.009891,-0.018668,0.045293,-0.042225,-0.015849,-0.002904,-0.023377,1,2
1,1a14,-0.016770,-0.016028,0.005558,-0.022389,0.048435,-0.029777,0.005205,-0.091042,0.002346,...,-0.027305,0.012193,0.013407,0.019745,-0.029758,-0.021011,0.000632,-0.014885,0,4
2,1a2y,0.022052,-0.024678,0.013147,-0.041079,0.054560,-0.009407,-0.007112,-0.076221,0.010534,...,-0.031617,-0.013962,-0.021564,0.045667,-0.028034,-0.023921,-0.020638,-0.019502,0,1
3,1a3l,-0.004759,0.000985,-0.010184,-0.027024,0.073321,-0.015843,0.003232,-0.090230,0.003318,...,-0.016519,0.000595,-0.000173,0.037635,-0.030517,-0.020022,-0.004965,-0.012199,0,4
4,1a4j,-0.008561,-0.015271,0.014696,-0.026142,0.075300,-0.019657,0.031890,-0.106666,-0.001778,...,-0.020049,-0.007701,-0.031986,0.017794,-0.022749,-0.019076,-0.008999,-0.030474,0,7


In [42]:
x_tap = pd.read_feather(path.join(DATA_DIR, "tap/embeddings/bert/bert_tap_embeddings.ftr"))
x_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,Abagovomab,-0.004248,-0.024501,-0.011330,-0.027170,0.062747,-0.024793,0.009313,-0.083316,-0.005339,...,-0.008674,-0.002387,-0.021563,0.001087,-0.020986,0.047104,-0.038736,-0.021780,-0.022153,-0.024539
1,Abituzumab,0.006593,-0.013591,-0.008454,-0.043601,0.065095,-0.016896,-0.001596,-0.090935,-0.002940,...,0.000486,-0.013063,-0.021852,-0.003531,-0.024620,0.027387,-0.041001,-0.025708,-0.016437,-0.034342
2,Abrilumab,0.019445,-0.002642,-0.011395,-0.058757,0.060623,-0.015046,0.006317,-0.083772,-0.006775,...,0.005834,-0.017189,-0.014557,0.003359,-0.035368,0.020287,-0.033941,-0.023549,-0.008720,-0.044038
3,Actoxumab,-0.006365,-0.043729,0.005978,-0.025613,0.067748,-0.009542,0.017723,-0.096801,-0.007752,...,-0.000024,-0.012986,-0.019753,-0.004326,-0.044124,0.019544,-0.039559,-0.015679,-0.008837,-0.043877
4,Adalimumab,-0.012995,-0.035269,0.014127,-0.042136,0.080592,-0.012831,0.031889,-0.091308,-0.018166,...,-0.002067,-0.022957,-0.021145,-0.002762,-0.058199,0.026781,-0.046815,-0.010110,-0.008785,-0.041761


In [43]:
try_all(x_chen_train.drop(["Ab_ID", "Y", "cluster_merged"], axis=1), x_chen_train["Y"], 
        x_chen_valid.drop(["Ab_ID", "Y"], axis=1),  x_chen_valid["Y"], 
        x_chen_train["cluster_merged"], "bert", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data bert 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
logistic_regression, bert
F1: 0.5853658536585366
MCC: 0.46689822169560774
Accuracy: 0.7866108786610879
Precision: 0.4864864864864865
Recall: 0.7346938775510204
AUC: 0.7673469387755102
-----


Training model random_forest on data bert 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
random_forest, bert
F1: 0.5299145299145299
MCC: 0.39184414279833696
Accuracy: 0.7698744769874477
Precision: 0.45588235294117646
Recall: 0.6326530612244898
AUC: 0.7189581095596134
-----


Training model gradient_boosting on data bert 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, bert
F1: 0.2903225806451613
MCC: 0.2894843826095956
Accuracy: 0.8158995815899581
Precision: 0.6923076923076923
Recall: 0.1836734693877551
AUC: 0.5813104189044039
-----


Training model SVM on data bert 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/

SVM, bert
F1: 0.6218487394957983
MCC: 0.5157875016520946
Accuracy: 0.8117154811715481
Precision: 0.5285714285714286
Recall: 0.7551020408163265
AUC: 0.7907089151450053
-----


Training model multilayer_perceptron on data bert 

Fitting 10 folds for each of 6 candidates, totalling 60 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


multilayer_perceptron, bert
F1: 0.5714285714285715
MCC: 0.49318530374023684
Accuracy: 0.8493723849372385
Precision: 0.6857142857142857
Recall: 0.4897959183673469
AUC: 0.715950590762621
-----


In [44]:
test_all(x_tap.drop("Ab_ID", axis=1), tap_data["Y"], "bert", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on bert...
Testing model random_forest on bert...
Testing model gradient_boosting on bert...
Testing model SVM on bert...
Testing model multilayer_perceptron on bert...


# SeqVec Embeddings

In [45]:
x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/seqvec/seqvec_chen_embeddings.ftr"))
x_chen_train = x_chen.merge(chen_train[["Antibody_ID", "Y", "cluster_merged"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_valid = x_chen.merge(chen_valid[["Antibody_ID", "Y"]], left_on="Ab_ID", right_on="Antibody_ID").drop("Antibody_ID", axis=1)
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,2040,2041,2042,2043,2044,2045,2046,2047,Y,cluster_merged
0,1a0q,0.073347,-0.186018,-0.233184,-0.165189,0.064193,-0.030149,0.043134,0.111651,-0.120727,...,0.138852,0.049886,-0.044254,-0.012993,-0.032100,-0.005154,-0.035896,-0.053616,1,2
1,1a14,0.066646,-0.216164,-0.275053,-0.124785,0.051032,-0.010701,0.042427,0.123428,-0.098271,...,0.141865,-0.114347,0.011103,-0.044319,-0.028007,0.020282,-0.050135,-0.095745,0,4
2,1a2y,0.089280,-0.191509,-0.165287,-0.087335,0.067743,-0.043121,0.027767,0.232575,-0.080467,...,0.216815,0.082564,0.016979,-0.003557,-0.081891,0.005624,-0.041633,-0.220336,0,1
3,1a3l,0.007294,-0.190514,-0.289291,-0.150268,0.056585,0.023869,-0.016217,0.094631,-0.208477,...,0.179691,-0.022877,0.003620,-0.055541,-0.035867,-0.030935,-0.006666,0.009036,0,4
4,1a4j,-0.016434,-0.144044,-0.292733,-0.124054,0.080313,-0.051480,0.000747,0.087235,-0.182320,...,0.267914,0.044216,0.020647,0.060805,-0.116175,0.111569,0.045578,-0.108075,0,7


In [46]:
x_tap = pd.read_feather(path.join(DATA_DIR, "tap/embeddings/seqvec/seqvec_tap_embeddings.ftr"))
x_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,Abagovomab,-0.004248,-0.024501,-0.011330,-0.027170,0.062747,-0.024793,0.009313,-0.083316,-0.005339,...,-0.008674,-0.002387,-0.021563,0.001087,-0.020986,0.047104,-0.038736,-0.021780,-0.022153,-0.024539
1,Abituzumab,0.006593,-0.013591,-0.008454,-0.043601,0.065095,-0.016896,-0.001596,-0.090935,-0.002940,...,0.000486,-0.013063,-0.021852,-0.003531,-0.024620,0.027387,-0.041001,-0.025708,-0.016437,-0.034342
2,Abrilumab,0.019445,-0.002642,-0.011395,-0.058757,0.060623,-0.015046,0.006317,-0.083772,-0.006775,...,0.005834,-0.017189,-0.014557,0.003359,-0.035368,0.020287,-0.033941,-0.023549,-0.008720,-0.044038
3,Actoxumab,-0.006365,-0.043729,0.005978,-0.025613,0.067748,-0.009542,0.017723,-0.096801,-0.007752,...,-0.000024,-0.012986,-0.019753,-0.004326,-0.044124,0.019544,-0.039559,-0.015679,-0.008837,-0.043877
4,Adalimumab,-0.012995,-0.035269,0.014127,-0.042136,0.080592,-0.012831,0.031889,-0.091308,-0.018166,...,-0.002067,-0.022957,-0.021145,-0.002762,-0.058199,0.026781,-0.046815,-0.010110,-0.008785,-0.041761


In [47]:
try_all(x_chen_train.drop(["Ab_ID", "Y", "cluster_merged"], axis=1), x_chen_train["Y"], 
        x_chen_valid.drop(["Ab_ID", "Y"], axis=1),  x_chen_valid["Y"], 
        x_chen_train["cluster_merged"], "seqvec", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data seqvec 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/braz

logistic_regression, seqvec
F1: 0.5925925925925926
MCC: 0.47840537736317834
Accuracy: 0.8158995815899581
Precision: 0.5423728813559322
Recall: 0.6530612244897959
AUC: 0.7554779806659505
-----


Training model random_forest on data seqvec 

Fitting 10 folds for each of 10 candidates, totalling 100 fits
random_forest, seqvec
F1: 0.49462365591397844
MCC: 0.37381609054520326
Accuracy: 0.803347280334728
Precision: 0.5227272727272727
Recall: 0.46938775510204084
AUC: 0.6794307196562835
-----


Training model gradient_boosting on data seqvec 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, seqvec
F1: 0.2571428571428572
MCC: 0.1718621031990959
Accuracy: 0.7824267782426778
Precision: 0.42857142857142855
Recall: 0.1836734693877551
AUC: 0.5602577873254565
-----


Training model SVM on data seqvec 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/

SVM, seqvec
F1: 0.5528455284552845
MCC: 0.4220654876718029
Accuracy: 0.7698744769874477
Precision: 0.4594594594594595
Recall: 0.6938775510204082
AUC: 0.7416756176154672
-----


Training model multilayer_perceptron on data seqvec 

Fitting 10 folds for each of 6 candidates, totalling 60 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


multilayer_perceptron, seqvec
F1: 0.4130434782608695
MCC: 0.27477895650740347
Accuracy: 0.7740585774058577
Precision: 0.4418604651162791
Recall: 0.3877551020408163
AUC: 0.6307196562835661
-----


In [48]:
test_all(x_tap.drop("Ab_ID", axis=1), tap_data["Y"], "seqvec", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on seqvec...
Testing model random_forest on seqvec...
Testing model gradient_boosting on seqvec...
Testing model SVM on seqvec...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing model multilayer_perceptron on seqvec...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# One-hot Encoding

In [57]:
x_heavy = pd.read_feather(path.join(DATA_DIR, "chen/abnumber/chen_heavy_one_hot.ftr")).set_index("index")
# rows 1921 and 2097 could not be encoded
x_light = pd.read_feather(path.join(DATA_DIR, "chen/abnumber/chen_light_one_hot.ftr")).set_index("Id")
x_chen = x_heavy.merge(x_light, left_index=True, right_index=True, suffixes=["_h", "_l"])
x_chen.index = x_heavy.index
train_idx = list(chen_train.index)
train_idx.remove(1921)
train_idx.remove(2097)
x_chen_train = x_chen.loc[train_idx]
x_chen_valid = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID_h,A_1_h,C_1_h,D_1_h,E_1_h,F_1_h,G_1_h,H_1_h,I_1_h,K_1_h,...,O_138_l,P_138_l,Q_138_l,R_138_l,S_138_l,T_138_l,U_138_l,V_138_l,W_138_l,Y_138_l
index,,,,,,,,,,,,,,,,,,,,,
2073,6aod,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1517,4yny,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025,5xcv,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2070,6and,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
666,2xqy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
train_labels = chen_train.loc[train_idx]["Y"]

In [60]:
# add groups for data split
try_all(x_chen_train.drop(["Ab_ID_h", "Ab_ID_l"], axis=1), train_labels, x_chen_valid.drop(["Ab_ID_h", "Ab_ID_l"], axis=1), chen_valid["Y"], "onehot", EVAL_DIR, preprocessing=None)



Training model logistic_regression on data onehot 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/home/braz

logistic_regression, onehot
F1: 0.3676470588235294
MCC: 0.15429345341033412
Accuracy: 0.6401673640167364
Precision: 0.28735632183908044
Recall: 0.5102040816326531
AUC: 0.5919441460794843
-----


Training model random_forest on data onehot 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
random_forest, onehot
F1: 0.27499999999999997
MCC: 0.14326352207226128
Accuracy: 0.7573221757322176
Precision: 0.3548387096774194
Recall: 0.22448979591836735
AUC: 0.5596133190118153
-----


Training model gradient_boosting on data onehot 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sk

gradient_boosting, onehot
F1: 0.18666666666666665
MCC: 0.05556759229181543
Accuracy: 0.7447698744769874
Precision: 0.2692307692307692
Recall: 0.14285714285714285
AUC: 0.5214285714285714
-----


Training model SVM on data onehot 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
SVM, onehot
F1: 0.3597122302158273
MCC: 0.14006354446702016
Accuracy: 0.6276150627615062
Precision: 0.2777777777777778
Recall: 0.5102040816326531
AUC: 0.5840494092373791
-----


Training model multilayer_perceptron on data onehot 

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


multilayer_perceptron, onehot
F1: 0.3157894736842105
MCC: 0.1464015398458587
Accuracy: 0.7280334728033473
Precision: 0.32608695652173914
Recall: 0.30612244897959184
AUC: 0.5714822771213749
-----


In [82]:
x_tap_heavy = pd.read_feather(path.join(DATA_DIR, "tap/abnumber/tap_heavy_one_hot.ftr")).drop(["index"], axis=1)
x_tap_light = pd.read_feather(path.join(DATA_DIR, "tap/abnumber/tap_light_one_hot.ftr")).drop(["index"], axis=1)
x_tap = x_tap_heavy.merge(x_tap_light, left_index=True, right_index=True, suffixes=["_h", "_l"]).drop("Ab_ID_l", axis=1)

In [83]:
x_tap

,Ab_ID_h,A_1_h,C_1_h,D_1_h,E_1_h,F_1_h,G_1_h,H_1_h,I_1_h,K_1_h,...,O_120_l,P_120_l,Q_120_l,R_120_l,S_120_l,T_120_l,U_120_l,V_120_l,W_120_l,Y_120_l
0,Abagovomab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abituzumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abrilumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Actoxumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adalimumab,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Visilizumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
237,Vonlerolizumab,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238,Zalutumumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,Zanolimumab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
test_all(x_tap.drop(["Ab_ID_h"], axis=1), tap_data["Y"], "onehot", EVAL_DIR, preprocessing=None)

Testing model logistic_regression on onehot...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- A_121
- A_122
- A_123
- A_124
- A_125
- ...
Feature names seen at fit time, yet now missing:
- A_121_h
- A_121_l
- A_122_h
- A_122_l
- A_123_h
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 5632 features, but LogisticRegression is expecting 7524 features as input.